In [4]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

# Read in and grayscale the image
image = mpimg.imread('exit-ramp.jpg')
gray = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)

# Define a kernel size and apply Gaussian smoothing
kernel_size = 5
blur_gray = cv2.GaussianBlur(gray,(kernel_size, kernel_size),0)

# Define our parameters for Canny and apply
low_threshold = 50
high_threshold = 150
edges = cv2.Canny(blur_gray, low_threshold, high_threshold)

# Next we'll create a masked edges image using cv2.fillPoly()
mask = np.zeros_like(edges)   
ignore_mask_color = 255   

# This time we are defining a four sided polygon to mask
imshape = image.shape
vertices = np.array([[(0,imshape[0]),(imshape[1]/2, 300), (imshape[1]/2+1, 300), (imshape[1],imshape[0])]], dtype=np.int32)
cv2.fillPoly(mask, vertices, ignore_mask_color)
masked_edges = cv2.bitwise_and(edges, mask)

# Define the Hough transform parameters
# Make a blank the same size as our image to draw on
rho = 1 # distance resolution in pixels of the Hough grid
theta = np.pi/180 # angular resolution in radians of the Hough grid
threshold = 15     # minimum number of votes (intersections in Hough grid cell)
min_line_length = 20 #minimum number of pixels making up a line
max_line_gap = 20    # maximum gap in pixels between connectable line segments
line_image = np.copy(image)*0 # creating a blank to draw lines on

# Run Hough on edge detected image
# Output "lines" is an array containing endpoints of detected line segments
lines = cv2.HoughLinesP(masked_edges, rho, theta, threshold, np.array([]),
                            min_line_length, max_line_gap)
#plt.imshow(line_image)
lines.shape
# Iterate over the output "lines" and draw lines on a blank image
for line in lines:
    for x1,y1,x2,y2 in line:
        cv2.line(line_image,(x1,y1),(x2,y2),(255,0,0),10)
#
# connect the line segement and also do the extrapolation
rm = np.array([])
rc = np.array([])
lm = np.array([])
lc = np.array([])
i_rightslope = 0
i_leftslope = 0
for line in lines:
    for x1,y1,x2,y2 in line:
        slope = (y2-y1)/(x2-x1)
        center = [(x2+x1)/2,(y2+y1)/2]
        if slope < 0:
            rm[i_rightslope] = slope
            rc[i_rightslope,:] = center
            i_rightslope = i_rightslope+1;
        else:
            rm[i_leftslope] = slope
            rc[i_leftslope,:] = center
            i_leftslope = i_leftslope+1;
            
r_slope = np.sum(rm)/len(rm)
l_slope = np.sum(lm)/len(lm)

r_center = np.divide(np.sum(rc,axis=0),len(rc))
l_center = np.divide(np.sum(lc,axis=0),len(lc))

imshape = img.shape
x1_l = (imshape[0]-l_center[1])/l_slope+l_center[0]
x2_l = (imshape[0]/2-l_center[1])/l_slope+l_center[0] 
x1_r = (imshape[0]-r_center[1])/r_slope+r_center[0]
x2_r = (imshape[0]/2-r_center[1])/r_slope+r_center[0] 
y1_l = imshape[0]
y2_l = imshape[0]/2
y1_r = imshape[0]
y2_r = imshape[0]/2
cv2.line(img, (x1_l, y1_l), (x2_l, y2_l), (x1_r, y1_r), (x2_r, y2_r), color, thickness)

IndexError: index 0 is out of bounds for axis 0 with size 0